# Lab 2 of Computationa Intelligence
### Ricardo Nicida Kazama

## Set Covering with Genetic Algorithm
### Task 
Given a number $N$ and some lists of integers $P = (L_0,L_1,L_2,...,L_N)$, determine, if possible, $S = (L_{S_0},L_{S_1},L_{S_2},...,L_{S_N})$ such that each number between $0$ and $N-1$ appears in at least one list
\begin{equation}
    \forall n \in [0,N-1] \ \exist i : n \in L_{S_i}
\end{equation}

### Problem generator

In [1]:
import logging
import random

In [2]:
def problem(N, seed=None):
    """Creates an instance of the problem"""

    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

### Solution

In [3]:
from itertools import compress
from collections import namedtuple

In [116]:
N = 5
POPULATION_SIZE = 10
OFFSPRING_SIZE = 2
GENERATIONS = 5
PROB = 0.5 # probability to choose 1 for each one of the locus in the population
Individual = namedtuple('Individual', ('genome', 'fitness','goal_reached', 'w'))

In [108]:
# this function evaluats the fitness and if the goal was reached
def fitness_goal_eval(list_of_lists, genome, goal):
    current_goal = goal
    solution = list(compress(list_of_lists, genome))
    # fitness = 0
    new_elements = 0
    repeated_elements = 0
    w = 0
    goal_reached = False

    if len(solution) == 0:
        return 0, False, 0
        
    for list_ in solution:
        list_length = len(list_)
        list_ = set(list_)
        cg_length = len(current_goal)
        current_goal = current_goal - list_
        cg_new_length = len(current_goal)

        # fitness += cg_length - cg_new_length   # new elements (positive)
        # fitness += (cg_length - cg_new_length) - list_length # repeated elements (negative)
        new_elements += cg_length - cg_new_length   # new elements
        repeated_elements += list_length - (cg_length - cg_new_length) # repeated elements

        w += list_length

    if cg_new_length == 0:
        goal_reached = True

    fitness = new_elements - repeated_elements

    return fitness, goal_reached, w


def generate_population(list_of_lists, goal):
    population = list()

    genomes = [tuple(random.choices([1, 0], weights=(PROB,1-PROB), k=len(list_of_lists))) for _ in range(POPULATION_SIZE)]

    for genome in genomes:
        fitness, goal_reached, w = fitness_goal_eval(list_of_lists, genome, goal)
        population.append(Individual(genome, fitness, goal_reached, w))
    return population


def select_parent(population, tournament_size=2):
    subset = random.choices(population, k=tournament_size)
    return max(subset, key=lambda i: i.fitness)


def cross_over(p1, p2, genome_size, list_of_lists, goal):
    g1, f1 = p1.genome, p1.fitness
    g2, f2 = p2.genome, p2.fitness
    cut = int((f1+1e-6)/(f1+f2+1e-6)*genome_size)
    ng1 = g1[:cut] + g2[cut:]
    # ng2 = g2[-cut:] + g1[:-cut]
    # _, f_gn1, _ = fitness_goal_eval(list_of_lists, ng1, goal)
    # _, f_gn2, _ = fitness_goal_eval(list_of_lists, ng2, goal)
    # if f_gn1 < f_gn2:
    #     return ng2
    return ng1


def mutation(g, genome_size, k=1):
    for _ in range(k):
        cut = random.randint(1, genome_size)
        if N < 20:
            ng = g[:cut-1] + (1-g[cut-1],) + g[cut:]
        else: 
            cut_size = int(genome_size*0.2)
            new_genome_cut = tuple(random.choices([1, 0], weights=(1, 39), k=2*cut_size))
            ng = g[:cut-1-cut_size] + new_genome_cut + g[cut+cut_size:]
    return ng

In [102]:
def genetic_algorithm():
    # create problem
    list_of_lists = problem(N, seed=42)
    genome_size = len(list_of_lists)
    goal = set(range(N))

    # create the population
    population = generate_population(list_of_lists, goal)

    for g in range(GENERATIONS):
        population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE-OFFSPRING_SIZE]

        for i in range(OFFSPRING_SIZE):
            p1 = select_parent(population, tournament_size=int(0.2*genome_size))
            p2 = select_parent(population, tournament_size=int(0.2*genome_size))
            o = cross_over(p1, p2, genome_size, list_of_lists, goal)
            fitness, goal_reached, w = fitness_goal_eval(list_of_lists, o, goal)
            o = mutation(o, genome_size, k=2)
                
            population.append(Individual(o, fitness, goal_reached, w))

        
    
    for i in population:
        if i.goal_reached:
            return i, population
    
    print(f"No solution for current population (N={N})")
    return None, population

In [112]:
N = 500
POPULATION_SIZE = 50
OFFSPRING_SIZE = 20
GENERATIONS = 200
PROB = 0.5

logging.getLogger().setLevel(logging.INFO)

solution, population = genetic_algorithm()
if solution != None:
    logging.info(
        f" Genetic algorithm solution for N={N:,}: "
        + f"fitness={solution.fitness:,} "
        + f"w={solution.w:,} "
        + f"(bloat={solution.w/N*100:.0f}%)"
    )


INFO:root: Genetic algorithm solution for N=500: fitness=-6,876 w=7,876 (bloat=1575%)


In [114]:
N = 500
POPULATION_SIZE = 50
OFFSPRING_SIZE = 20
GENERATIONS = 200
PROB = 0.5

logging.getLogger().setLevel(logging.INFO)

for N in [5, 10, 20, 100, 500, 1000]:
    solution, population = genetic_algorithm()
    if solution != None:
        logging.info(
            f" Genetic algorithm solution for N={N:,}: "
            + f"fitness={solution.fitness:,} "
            + f"w={solution.w:,} "
            + f"(bloat={solution.w/N*100:.0f}%)"
        )

INFO:root: Genetic algorithm solution for N=5: fitness=5 w=5 (bloat=100%)
INFO:root: Genetic algorithm solution for N=10: fitness=9 w=11 (bloat=110%)


No solution for current population (N=20)


INFO:root: Genetic algorithm solution for N=100: fitness=-124 w=324 (bloat=324%)
INFO:root: Genetic algorithm solution for N=500: fitness=-6,876 w=7,876 (bloat=1575%)
INFO:root: Genetic algorithm solution for N=1,000: fitness=-183,263 w=185,263 (bloat=18526%)
